In [1]:
!git clone https://github.com/pl2599/GGWP-Toxic-Behavior.git

Cloning into 'GGWP-Toxic-Behavior'...
remote: Enumerating objects: 717, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 717 (delta 21), reused 38 (delta 15), pack-reused 668
Receiving objects: 100% (717/717), 297.88 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (224/224), done.


In [12]:
import os
import json
import time
import pandas as pd
import openai
from dotenv import load_dotenv
import tqdm

In [2]:
load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
df = pd.read_csv('GGWP-Toxic-Behavior/data/labeled/combined.csv')

In [5]:
df.head()

,index,text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,real faker tweet,0,0,0,0,0,0
1,1,Lmfao,0,0,0,0,0,0
2,2,"bruh, the best teemo player from my country is...",0,0,0,0,0,0
3,3,me and nunu tiered 3 simped for adc,0,0,0,0,0,0
4,4,yeah but i think since seraphine was released ...,0,0,0,0,0,0


In [6]:
df['label'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()

In [7]:
df['label'] = df['label'].apply(
    lambda x: 'clean' if x == [0, 0, 0, 0, 0, 0] else 'toxic' if x == [1, 0, 0, 0, 0, 0] else 'severe_toxic' if x == [0,
                                                                                                                      1,
                                                                                                                      0,
                                                                                                                      0,
                                                                                                                      0,
                                                                                                                      0] else 'obscene' if x == [
        0, 0, 1, 0, 0, 0] else 'threat' if x == [0, 0, 0, 1, 0, 0] else 'insult' if x == [0, 0, 0, 0, 1,
                                                                                          0] else 'identity_hate')

In [22]:
with open('prompts/original_prompt.txt', 'r') as f:
    original_prompt = f.read()

if not os.path.exists('outputs'):
    os.makedirs('outputs')

In [34]:
def sleep(seconds: int) -> None:
    """Sleep for a given number of seconds."""
    time.sleep(seconds)

In [35]:
def chat_model(prompt: str) -> str:
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    except openai.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
        sleep(3)
        return chat_model(prompt)
    except openai.error.Timeout as e:
        print(f"OpenAI API request timed out: {e}")
        sleep(3)
        return chat_model(prompt)
    except openai.error.APIConnectionError as e:
        print(f"OpenAI API request failed to connect: {e}")
        sleep(3)
        return chat_model(prompt)
    except openai.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
        sleep(5)
        return chat_model(prompt)
    except Exception as e:
        print(f"Unexpected error: {e}")

In [43]:
def parse_json_output(raw_text: str) -> dict:
    """Parse the raw text output from OpenAI API into a dictionary."""
    if "```json" in raw_text:
        raw_text = raw_text.split("```json")[1].split("```")[0]
    try:
        return json.loads(raw_text, strict=False)
    except Exception as e:
        print(f"Unexpected error: {e}")

In [46]:
if not os.path.exists('outputs/parsed'):
    os.makedirs('outputs/parsed')

if not os.path.exists('outputs/raw'):
    os.makedirs('outputs/raw')

In [45]:
prompt = original_prompt
prompt_version = 'v1'

for index, message in zip(df['index'], df['text']):
    if os.path.exists(f'outputs/raw/{prompt_version}/{index}.txt') and os.path.exists(
            f'outputs/parsed/{prompt_version}/{index}.json'):
        continue

    response = chat_model(prompt.replace('<message>', message))
    with open(f'outputs/raw/{prompt_version}/{index}.txt', 'w') as f:
        f.write(response)

    parsed_response = parse_json_output(response)
    with open(f'outputs/parsed/{prompt_version}/{index}.json', 'w') as f:
        parsed_response['prompt_version'] = prompt_version
        parsed_response['label'] = df[df['index'] == index]['label'].values[0]
        json.dump(parsed_response, f, indent=2)

    sleep(5)

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/raw/v1/0.txt'

In [ ]:
def accuracy_calculation(prompt_version: str):
    

In [ ]:
# TODO: Compare performance optimization via GPT-3.5 and GPT-4 and base performance of GPT-3.5 and GPT-4 without optimization
# TODO: Save raw data, logging, add resume-ability, handle OpenAI error
# 1. Load the prompt
# 2. Predict the result
# 3. Evaluate (accuracy)
# 4. Sample the result for prompt optimization
# 5. Get new prompt
# 6. Loop until reach specified n or acc or stagnant

# Main prompt: Replace "<message>"
# Modifying prompt: Replace: "<sample>"
# Each entry in <sample>
# |- Message: <message>
# |- Predicted class: <prediction>
# |- Predicted reason: <reason>
# |- Ground truth: <ground_truth>